In [70]:
sc

In [24]:

from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from string import split,strip

### Higgs data set
* **URL:** http://archive.ics.uci.edu/ml/datasets/HIGGS#  
* **Abstract:** This is a classification problem to distinguish between a signal process which produces Higgs bosons and a background process which does not.

**Data Set Information:**  
The data has been produced using Monte Carlo simulations. The first 21 features (columns 2-22) are kinematic properties measured by the particle detectors in the accelerator. The last seven features are functions of the first 21 features; these are high-level features derived by physicists to help discriminate between the two classes. There is an interest in using deep learning methods to obviate the need for physicists to manually develop such features. Benchmark results using Bayesian Decision Trees from a standard physics package and 5-layer neural networks are presented in the original paper. The last 500,000 examples are used as a test set.



In [67]:
feature_text='lepton pT, lepton eta, lepton phi, missing energy magnitude, missing energy phi, jet 1 pt, jet 1 eta, jet 1 phi, jet 1 b-tag, jet 2 pt, jet 2 eta, jet 2 phi, jet 2 b-tag, jet 3 pt, jet 3 eta, jet 3 phi, jet 3 b-tag, jet 4 pt, jet 4 eta, jet 4 phi, jet 4 b-tag, m_jj, m_jjj, m_lv, m_jlv, m_bb, m_wbb, m_wwbb'
features=[strip(a) for a in split(feature_text,',')]
print len(features),features

28 ['lepton pT', 'lepton eta', 'lepton phi', 'missing energy magnitude', 'missing energy phi', 'jet 1 pt', 'jet 1 eta', 'jet 1 phi', 'jet 1 b-tag', 'jet 2 pt', 'jet 2 eta', 'jet 2 phi', 'jet 2 b-tag', 'jet 3 pt', 'jet 3 eta', 'jet 3 phi', 'jet 3 b-tag', 'jet 4 pt', 'jet 4 eta', 'jet 4 phi', 'jet 4 b-tag', 'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb']


In [47]:
%cd ~/BigData/Data/

/Users/yoavfreund/academic.papers/Courses/BigDataAnalytics/BigData_spring2016/Data


In [48]:
!mkdir HIGGS
%cd HIGGS

mkdir: HIGGS: File exists
/Users/yoavfreund/academic.papers/Courses/BigDataAnalytics/BigData_spring2016/Data/HIGGS


In [6]:
!curl -O http://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 47 2685M   47 1262M    0     0  9796k      0  0:04:40  0:02:12  0:02:28  9.9M^C


In [ ]:
!gunzip HIGGS.csv.gz ## Takes 5-10 minutes

In [60]:
!head -2 HIGGS.csv
%pwd

1.000000000000000000e+00,8.692932128906250000e-01,-6.350818276405334473e-01,2.256902605295181274e-01,3.274700641632080078e-01,-6.899932026863098145e-01,7.542022466659545898e-01,-2.485731393098831177e-01,-1.092063903808593750e+00,0.000000000000000000e+00,1.374992132186889648e+00,-6.536741852760314941e-01,9.303491115570068359e-01,1.107436060905456543e+00,1.138904333114624023e+00,-1.578198313713073730e+00,-1.046985387802124023e+00,0.000000000000000000e+00,6.579295396804809570e-01,-1.045456994324922562e-02,-4.576716944575309753e-02,3.101961374282836914e+00,1.353760004043579102e+00,9.795631170272827148e-01,9.780761599540710449e-01,9.200048446655273438e-01,7.216574549674987793e-01,9.887509346008300781e-01,8.766783475875854492e-01
1.000000000000000000e+00,9.075421094894409180e-01,3.291472792625427246e-01,3.594118654727935791e-01,1.497969865798950195e+00,-3.130095303058624268e-01,1.095530629158020020e+00,-5.575249195098876953e-01,-1.588229775428771973e+00,2.173076152801513672e+00,8.12581181526

u'/Users/yoavfreund/academic.papers/Courses/BigDataAnalytics/BigData_spring2016/Data/HIGGS'

In [77]:
F=sc.textFile(u'/Users/yoavfreund/academic.papers/Courses/BigDataAnalytics/BigData_spring2016/Data/HIGGS/HIGGS.csv')

In [78]:
F.first()

u'1.000000000000000000e+00,8.692932128906250000e-01,-6.350818276405334473e-01,2.256902605295181274e-01,3.274700641632080078e-01,-6.899932026863098145e-01,7.542022466659545898e-01,-2.485731393098831177e-01,-1.092063903808593750e+00,0.000000000000000000e+00,1.374992132186889648e+00,-6.536741852760314941e-01,9.303491115570068359e-01,1.107436060905456543e+00,1.138904333114624023e+00,-1.578198313713073730e+00,-1.046985387802124023e+00,0.000000000000000000e+00,6.579295396804809570e-01,-1.045456994324922562e-02,-4.576716944575309753e-02,3.101961374282836914e+00,1.353760004043579102e+00,9.795631170272827148e-01,9.780761599540710449e-01,9.200048446655273438e-01,7.216574549674987793e-01,9.887509346008300781e-01,8.766783475875854492e-01'

In [88]:
def LineToLabeledPoint(line):
    V = [float(x) for x in split(line,',')]
    return LabeledPoint(V[0],V[1:])
Data=F.map(LineToLabeledPoint)
Data.take(2)

[LabeledPoint(1.0, [0.869293212891,-0.635081827641,0.22569026053,0.327470064163,-0.689993202686,0.754202246666,-0.24857313931,-1.09206390381,0.0,1.37499213219,-0.653674185276,0.930349111557,1.10743606091,1.13890433311,-1.57819831371,-1.0469853878,0.0,0.65792953968,-0.0104545699432,-0.0457671694458,3.10196137428,1.35376000404,0.979563117027,0.978076159954,0.920004844666,0.721657454967,0.988750934601,0.876678347588]),
 LabeledPoint(1.0, [0.907542109489,0.329147279263,0.359411865473,1.4979698658,-0.313009530306,1.09553062916,-0.55752491951,-1.58822977543,2.1730761528,0.812581181526,-0.213641926646,1.27101457119,2.21487212181,0.499993950129,-1.26143181324,0.732156157494,0.0,0.398700892925,-1.13893008232,-0.000819110195152,0.0,0.302219897509,0.833048164845,0.985699653625,0.978098392487,0.779732167721,0.992355763912,0.798342585564])]

In [34]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils

In [90]:
(trainingData,testData)=Data.randomSplit([0.7,0.3])

In [91]:
model=GradientBoostedTrees.trainClassifier(trainingData,categoricalFeaturesInfo={},numIterations=3)

In [95]:
print model.toDebugString()

TreeEnsembleModel classifier with 3 trees

  Tree 0:
    If (feature 25 <= 1.0861190557479858)
     If (feature 25 <= 0.6328302025794983)
      If (feature 27 <= 0.8756244778633118)
       Predict: -0.011497738904384089
      Else (feature 27 > 0.8756244778633118)
       Predict: -0.3049196965656206
     Else (feature 25 > 0.6328302025794983)
      If (feature 26 <= 0.797487199306488)
       Predict: -0.03303820050523515
      Else (feature 26 > 0.797487199306488)
       Predict: 0.40157295930122033
    Else (feature 25 > 1.0861190557479858)
     If (feature 22 <= 1.0201389789581299)
      If (feature 27 <= 1.0021007061004639)
       Predict: -0.2633900638217995
      Else (feature 27 > 1.0021007061004639)
       Predict: -0.6008335670180185
     Else (feature 22 > 1.0201389789581299)
      If (feature 25 <= 1.7949312925338745)
       Predict: 0.015490085191346656
      Else (feature 25 > 1.7949312925338745)
       Predict: -0.3979820824348253
  Tree 1:
    If (feature 27 <= 0.92459601

### Cover Type
* **URL:** http://archive.ics.uci.edu/ml/datasets/Covertype
* **Abstract:** Forest CoverType dataset
* **Data Set Description:** http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.info

In [1]:
%cd ~/BigData/Data/
!ls -lrt | tail

/Users/yoavfreund/academic.papers/Courses/BigDataAnalytics/BigData_spring2016/Data
-rw-r--r--   1 yoavfreund  staff        25 Apr 24 11:05 hw2-files.txt
-rw-r--r--   1 yoavfreund  staff        31 Apr 24 11:05 hw2-files-5gb.txt
-rw-r--r--   1 yoavfreund  staff        90 Apr 24 11:05 hw2-files-20gb.txt
-rw-r--r--   1 yoavfreund  staff        13 Apr 24 11:05 hw2-files-1gb.txt
-rw-r--r--   1 yoavfreund  staff        43 Apr 24 11:05 example.csv
drwxr-xr-x   3 yoavfreund  staff       102 Apr 24 11:47 regression
drwxr-xr-x  16 yoavfreund  staff       544 Apr 27 19:41 Weather
-rw-r--r--   1 yoavfreund  staff  26877621 Apr 30 12:15 STAT_TAVG_RANGE.pickle
drwxr-xr-x   4 yoavfreund  staff       136 May  9 11:30 HIGGS
drwxr-xr-x   3 yoavfreund  staff       102 May  9 11:46 covtype


In [2]:
!mkdir covtype
%cd covtype
!ls -l

mkdir: covtype: File exists
/Users/yoavfreund/academic.papers/Courses/BigDataAnalytics/BigData_spring2016/Data/covtype
total 146816
-rw-r--r--  1 yoavfreund  staff  75169317 May  9 11:46 covtype.data


In [3]:
!curl -O http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.7M  100 10.7M    0     0  3844k      0  0:00:02  0:00:02 --:--:-- 3843k


In [6]:
!gunzip -f covtype.data.gz
!ls -l

gunzip: can't stat: covtype.data.gz (covtype.data.gz.gz): No such file or directory
total 146816
-rw-r--r--  1 yoavfreund  staff  75169317 May  9 11:46 covtype.data


In [3]:
cols_txt="""
Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology,
Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways,
Hillshade_9am, Hillshade_Noon, Hillshade_3pm,
Horizontal_Distance_To_Fire_Points, Wilderness_Area (4 binarycolumns), 
Soil_Type (40 binary columns), Cover_Type
"""

In [4]:
from string import split,strip
cols=[strip(a) for a in split(cols_txt,',')]
colDict={a:[a] for a in cols}
colDict['Soil_Type (40 binary columns)'] = ['ST_'+str(i) for i in range(40)]
colDict['Wilderness_Area (4 binarycolumns)'] = ['WA_'+str(i) for i in range(4)]
Columns=[]
for item in cols:
    Columns=Columns+colDict[item]
print Columns

['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'WA_0', 'WA_1', 'WA_2', 'WA_3', 'ST_0', 'ST_1', 'ST_2', 'ST_3', 'ST_4', 'ST_5', 'ST_6', 'ST_7', 'ST_8', 'ST_9', 'ST_10', 'ST_11', 'ST_12', 'ST_13', 'ST_14', 'ST_15', 'ST_16', 'ST_17', 'ST_18', 'ST_19', 'ST_20', 'ST_21', 'ST_22', 'ST_23', 'ST_24', 'ST_25', 'ST_26', 'ST_27', 'ST_28', 'ST_29', 'ST_30', 'ST_31', 'ST_32', 'ST_33', 'ST_34', 'ST_35', 'ST_36', 'ST_37', 'ST_38', 'ST_39', 'Cover_Type']


In [5]:
!tail -2 covtype.data

2384,170,15,60,5,90,230,245,143,864,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
2383,165,13,60,4,67,231,244,141,875,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


In [12]:
import pandas as pd
df=pd.read_csv(open('covtype.data','r'),names=Columns,header=None)
df.head(1)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,ST_31,ST_32,ST_33,ST_34,ST_35,ST_36,ST_37,ST_38,ST_39,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5


In [15]:
%pylab inline

df.hist(bins=50,figsize=(16,16));

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [6]:
df.columns

NameError: name 'df' is not defined

In [12]:
%pwd

u'/Users/yoavfreund/academic.papers/Courses/BigDataAnalytics/BigData_spring2016/Data/covtype'

In [16]:
path='/Users/yoavfreund/academic.papers/Courses/BigDataAnalytics/BigData_spring2016/Data/covtype/covtype.data'
inputRDD=sc.textFile(path)
inputRDD

MapPartitionsRDD[9] at textFile at NativeMethodAccessorImpl.java:-2

In [27]:
inputRDD.first()

u'2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5'

In [28]:
Data=inputRDD.map(lambda line: [float(x) for x in line.split(',')])\
    .map(lambda V:LabeledPoint(V[-1],V[:-1]))
Data.first()

LabeledPoint(5.0, [2596.0,51.0,3.0,258.0,0.0,510.0,221.0,232.0,148.0,6279.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [33]:
(trainingData,testData)=Data.randomSplit([0.7,0.3])

### Gradient-Boosted Decision trees
Documentation: http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#module-pyspark.mllib.tree

In [35]:
model=GradientBoostedTrees.trainClassifier(trainingData,categoricalFeaturesInfo={},numIterations=3)

In [36]:
print model.toDebugString()

TreeEnsembleModel classifier with 3 trees

  Tree 0:
    If (feature 0 <= 2558.0)
     If (feature 10 <= 0.0)
      If (feature 0 <= 2471.0)
       Predict: 6.770351720454719
      Else (feature 0 > 2471.0)
       Predict: 5.740438796307397
     Else (feature 10 > 0.0)
      If (feature 9 <= 4578.0)
       Predict: 2.9670912951167727
      Else (feature 9 > 4578.0)
       Predict: 4.136960600375234
    Else (feature 0 > 2558.0)
     If (feature 0 <= 3317.0)
      If (feature 0 <= 2918.0)
       Predict: 3.3723028053456883
      Else (feature 0 > 2918.0)
       Predict: 2.1607261897547336
     Else (feature 0 > 3317.0)
      If (feature 12 <= 0.0)
       Predict: 4.217128874388255
      Else (feature 12 > 0.0)
       Predict: 7.578162967762839
  Tree 1:
    If (feature 0 <= 2960.0)
     If (feature 0 <= 2921.0)
      If (feature 0 <= 2898.0)
       Predict: 4.090717211786396E-4
      Else (feature 0 > 2898.0)
       Predict: 0.002271467388852949
     Else (feature 0 > 2921.0)
      If (